In [2]:
import sys
sys.path.append('/home/ec2-user/SageMaker')
import toniutils as tu

In [3]:
from keras.models import model_from_json

Using TensorFlow backend.
/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


<h2>Load Models</h2>

In [4]:
import numpy as np
import io
import pickle
import boto3
from keras.models import model_from_json

s3_client = boto3.client('s3')
#result = client.get_object(Bucket='bme-bucket', Key='models/keras-model-1.json')
#model = model_from_json(result["Body"].read())

In [5]:
models_dict ={}
for model in ['model-' + str(i) for i in range(1, 19)]:
    #print('models/keras-' + model + '.json')
    model_json = s3_client.get_object(Bucket='bme-bucket', Key='models/keras-' + model + '.json')
    models_dict[model] = model_from_json(model_json['Body'].read())

Instructions for updating:
Colocations handled automatically by placer.


<h2>Load Data</h2>

In [6]:
# download without using disk
my_array_data2 = io.BytesIO()
s3_client.download_fileobj('bme-bucket', 'engineered_data/Y_minmaxscaled.pkl', my_array_data2)
my_array_data2.seek(0)
Y = pickle.load(my_array_data2)

my_array_data2 = io.BytesIO()
s3_client.download_fileobj('bme-bucket', 'engineered_data/X.pkl', my_array_data2)
my_array_data2.seek(0)
X = pickle.load(my_array_data2)

<h3>Format data</h3>

In [7]:
X_2nd_layer = models_dict['model-1'].predict(X)

In [8]:
for i in range(2, 19):
    X_2nd_layer = np.hstack( (X_2nd_layer, models_dict['model-' + str(i)].predict(X)) )

<h2>2nd Layer Models</h2>

In [11]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV

from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor

from sklearn.tree import DecisionTreeRegressor

from sklearn.multioutput import MultiOutputRegressor, RegressorChain
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse

In [12]:
ESTIMATORS = {
    "Extra trees": ExtraTreesRegressor(n_estimators=10,
                                       max_features=32,     # Out of 20000
                                       random_state=0),
    #"K-nn": KNeighborsRegressor(), # To Lazy
    "Linear regression": LinearRegression(),
    "Ridge": RidgeCV(),
    "Lasso": Lasso(),
    "ElasticNet": ElasticNet(random_state=0),
    "RandomForestRegressor": RandomForestRegressor(max_depth=4, random_state=2),
    "Decision Tree Regressor":DecisionTreeRegressor(max_depth=5),
    "MultiO/P GBR": MultiOutputRegressor(GradientBoostingRegressor(n_estimators=5)),
    "MultiO/P AdaB": MultiOutputRegressor(AdaBoostRegressor(n_estimators=5)),
    "Regressor/C GBR": RegressorChain(GradientBoostingRegressor(n_estimators=5)),
    "Regressor/C AdaB": RegressorChain(AdaBoostRegressor(n_estimators=5))
}

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X_2nd_layer, Y, test_size = 0.2, random_state = 2)

In [15]:
FITTED_ESTIMATORS = {}
y_mse = {}

for name, estimator in ESTIMATORS.items():
    print(name)
    FITTED_ESTIMATORS[name] = estimator.fit(X_train, y_train)  # fit() with instantiated object
    y_mse[name] = mse(y_test, estimator.predict(X_test))

Extra trees
Linear regression
Ridge


MemoryError: 

In [47]:
lin_reg = LinearRegression()

In [48]:
lin_reg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [53]:
mse(lin_reg.predict(X_test), y_test)

0.02404002845175953

array([[ 6.71875000e-01,  3.88671875e-01,  1.73828125e-01,
         4.02343750e-01,  1.71875000e-01,  3.87695312e-01,
         1.66015625e-01,  4.09179688e-01,  1.78710938e-01,
        -3.66210938e-04,  6.09375000e-01,  3.59375000e-01,
         2.26562500e-01,  3.71093750e-01,  2.40234375e-01,
         3.35937500e-01,  2.26074219e-01,  3.67187500e-01,
         1.79687500e-01,  1.83105469e-04,  6.64062500e-01,
         4.04785156e-01,  3.90625000e-01,  4.37500000e-01,
         3.64257812e-01,  4.29687500e-01,  3.59375000e-01,
         3.75000000e-01,  3.16406250e-01,  5.49316406e-04,
         6.13281250e-01,  3.51562500e-01,  3.17382812e-02,
         3.37890625e-01,  3.18603516e-02,  3.46679688e-01,
         3.25927734e-02,  3.43750000e-01,  3.32031250e-02,
         1.83105469e-04,  6.50390625e-01,  3.87207031e-01,
         4.00390625e-02,  3.69140625e-01,  3.45458984e-02,
         3.80371094e-01,  3.66210938e-02,  3.91601562e-01,
         4.10156250e-02,  7.32421875e-04,  6.50390625e-0